# COGS 118A Final Project Fall 2024

## Dataset 2: [Estimation of Obesity Levels Based On Eating Habits and Physical Condition](https://archive.ics.uci.edu/dataset/544/estimation+of+obesity+levels+based+on+eating+habits+and+physical+condition)


In [1]:
import pandas as pd
import numpy as np
import scipy
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from ucimlrepo import fetch_ucirepo

# fetch dataset
estimation_of_obesity_levels_based_on_eating_habits_and_physical_condition = fetch_ucirepo(
    id=544)

# data (as pandas dataframes)
dataset = estimation_of_obesity_levels_based_on_eating_habits_and_physical_condition.data.original

dataset.head()

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
0,Female,21.0,1.62,64.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,0.0,1.0,no,Public_Transportation,Normal_Weight
1,Female,21.0,1.52,56.0,yes,no,3.0,3.0,Sometimes,yes,3.0,yes,3.0,0.0,Sometimes,Public_Transportation,Normal_Weight
2,Male,23.0,1.80,77.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,2.0,1.0,Frequently,Public_Transportation,Normal_Weight
3,Male,27.0,1.80,87.0,no,no,3.0,3.0,Sometimes,no,2.0,no,2.0,0.0,Frequently,Walking,Overweight_Level_I
4,Male,22.0,1.78,89.8,no,no,2.0,1.0,Sometimes,no,2.0,no,0.0,0.0,Sometimes,Public_Transportation,Overweight_Level_II


In [3]:
dataset.shape

(2111, 17)

In [4]:
dataset.dtypes

Gender                             object
Age                               float64
Height                            float64
Weight                            float64
family_history_with_overweight     object
FAVC                               object
FCVC                              float64
NCP                               float64
CAEC                               object
SMOKE                              object
CH2O                              float64
SCC                                object
FAF                               float64
TUE                               float64
CALC                               object
MTRANS                             object
NObeyesdad                         object
dtype: object

In [5]:
dataset.isnull().sum()

Gender                            0
Age                               0
Height                            0
Weight                            0
family_history_with_overweight    0
FAVC                              0
FCVC                              0
NCP                               0
CAEC                              0
SMOKE                             0
CH2O                              0
SCC                               0
FAF                               0
TUE                               0
CALC                              0
MTRANS                            0
NObeyesdad                        0
dtype: int64

In [6]:
dataset.describe()

,Age,Height,Weight,FCVC,NCP,CH2O,FAF,TUE
count,2111.000000,2111.000000,2111.000000,2111.000000,2111.000000,2111.000000,2111.000000,2111.000000
mean,24.312600,1.701677,86.586058,2.419043,2.685628,2.008011,1.010298,0.657866
std,6.345968,0.093305,26.191172,0.533927,0.778039,0.612953,0.850592,0.608927
min,14.000000,1.450000,39.000000,1.000000,1.000000,1.000000,0.000000,0.000000
25%,19.947192,1.630000,65.473343,2.000000,2.658738,1.584812,0.124505,0.000000
50%,22.777890,1.700499,83.000000,2.385502,3.000000,2.000000,1.000000,0.625350
75%,26.000000,1.768464,107.430682,3.000000,3.000000,2.477420,1.666678,1.000000
max,61.000000,1.980000,173.000000,3.000000,4.000000,3.000000,3.000000,2.000000


In [7]:
dataset['NObeyesdad'].value_counts()

NObeyesdad
Obesity_Type_I         351
Obesity_Type_III       324
Obesity_Type_II        297
Overweight_Level_I     290
Overweight_Level_II    290
Normal_Weight          287
Insufficient_Weight    272
Name: count, dtype: int64

In [8]:
dataset['NObeyesdad_binary'] = dataset['NObeyesdad'].apply(lambda x: 1 if x in [
                                                           'Obesity_Type_I', 'Obesity_Type_II', 'Obesity_Type_III', 'Overweight_Level_I', 'Overweight_Level_II'] else 0)

# Check the new distribution
print(dataset['NObeyesdad_binary'].value_counts())

NObeyesdad_binary
1    1552
0     559
Name: count, dtype: int64


# Pre Processing


In [9]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from tqdm import tqdm

In [10]:
categorical_columns = ['Gender', 'family_history_with_overweight',
                       'FAVC', 'CAEC', 'SMOKE', 'SCC', 'CALC', 'MTRANS']
numerical_columns = ['Age', 'Height', 'Weight', 'NCP', 'CH2O', 'FAF', 'TUE']

# Define the preprocessing steps
numerical_transformer = Pipeline(steps=[
    # Fill missing values with the mean
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())  # Standardize the numerical columns
])

categorical_transformer = Pipeline(steps=[
    # Fill missing values with the most frequent value
    ('imputer', SimpleImputer(strategy='most_frequent')),
    # One-hot encode categorical variables
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

# Combine the transformers into a single ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_columns),
        ('cat', categorical_transformer, categorical_columns)
    ]
)

# Drop the target column(s) and any irrelevant ones
X = dataset.drop(columns=['NObeyesdad', 'NObeyesdad_binary'])
y = dataset['NObeyesdad_binary']  # Target variable for binary classification

In [25]:
X

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS
0,Female,21.000000,1.620000,64.000000,yes,no,2.0,3.0,Sometimes,no,2.000000,no,0.000000,1.000000,no,Public_Transportation
1,Female,21.000000,1.520000,56.000000,yes,no,3.0,3.0,Sometimes,yes,3.000000,yes,3.000000,0.000000,Sometimes,Public_Transportation
2,Male,23.000000,1.800000,77.000000,yes,no,2.0,3.0,Sometimes,no,2.000000,no,2.000000,1.000000,Frequently,Public_Transportation
3,Male,27.000000,1.800000,87.000000,no,no,3.0,3.0,Sometimes,no,2.000000,no,2.000000,0.000000,Frequently,Walking
4,Male,22.000000,1.780000,89.800000,no,no,2.0,1.0,Sometimes,no,2.000000,no,0.000000,0.000000,Sometimes,Public_Transportation
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2106,Female,20.976842,1.710730,131.408528,yes,yes,3.0,3.0,Sometimes,no,1.728139,no,1.676269,0.906247,Sometimes,Public_Transportation
2107,Female,21.982942,1.748584,133.742943,yes,yes,3.0,3.0,Sometimes,no,2.005130,no,1.341390,0.599270,Sometimes,Public_Transportation
2108,Female,22.524036,1.752206,133.689352,yes,yes,3.0,3.0,Sometimes,no,2.054193,no,1.414209,0.646288,Sometimes,Public_Transportation
2109,Female,24.361936,1.739450,133.346641,yes,yes,3.0,3.0,Sometimes,no,2.852339,no,1.139107,0.586035,Sometimes,Public_Transportation


In [11]:
def comprehensive_model_evaluation(X, y, classifier, param_grid, split_ratios=[0.2, 0.5, 0.8]):
    """
    Comprehensive model evaluation across multiple splits and configurations
    """
    all_results = []

    for test_size in tqdm(split_ratios, desc="Split Ratios"):
        for seed in tqdm(range(3), desc="Random Seeds", leave=False):
            X_train, X_test, y_train, y_test = train_test_split(
                X, y, test_size=test_size, random_state=42 + seed)

            pipeline = Pipeline([
                ('preprocessor', preprocessor),
                ('classifier', classifier)
            ])

            grid_search = GridSearchCV(
                pipeline,
                param_grid,
                cv=5,
                scoring='accuracy',
                n_jobs=-1
            )
            grid_search.fit(X_train, y_train)

            best_model = grid_search.best_estimator_
            train_pred = best_model.predict(X_train)
            test_pred = best_model.predict(X_test)

            result = {
                'Test Size': f"{int((1-test_size)*100)}-{int(test_size*100)}",
                'Random Seed': seed,
                'Best Params': str(grid_search.best_params_),
                'Best CV Score': grid_search.best_score_,
                'Train Accuracy': accuracy_score(y_train, train_pred),
                'Test Accuracy': accuracy_score(y_test, test_pred),
                'Classifier': type(classifier).__name__
            }

            report = classification_report(y_test, test_pred, output_dict=True)
            result.update({
                'Macro Precision': report['macro avg']['precision'],
                'Macro Recall': report['macro avg']['recall'],
                'Macro F1-Score': report['macro avg']['f1-score']
            })

            # Add confusion matrix
            cm = confusion_matrix(y_test, test_pred)
            result['Confusion Matrix'] = cm

            all_results.append(result)

    results_df = pd.DataFrame(all_results)
    print("Evaluation Summary:")
    print(results_df.groupby(['Test Size', 'Classifier'])[
          ['Train Accuracy', 'Test Accuracy', 'Best CV Score']].agg(['mean', 'std']))

    # Optionally, you could print confusion matrices for all splits
    for result in all_results:
        print(f"Confusion Matrix for Test Size {
              result['Test Size']} and Seed {result['Random Seed']}:")
        print(result['Confusion Matrix'])

    return results_df

## Logistic Regression


In [12]:
lr_param_grid = {
    'classifier__C': [0.001, 0.01, 0.1, 1, 10, 100],
    'classifier__penalty': ['l1', 'l2'],
    'classifier__solver': ['liblinear']
}

# Comprehensive evaluation
lr_results = comprehensive_model_evaluation(
    X, y,
    LogisticRegression(),
    lr_param_grid
)

lr_results

Split Ratios:   0%|          | 0/3 [00:00<?, ?it/s]

Split Ratios: 100%|██████████| 3/3 [00:05<00:00,  1.93s/it]

Evaluation Summary:
                             Train Accuracy           Test Accuracy            \
                                       mean       std          mean       std   
Test Size Classifier                                                            
19-80     LogisticRegression       0.996051  0.006841      0.987764  0.002670   
50-50     LogisticRegression       0.997472  0.001448      0.991477  0.000947   
80-20     LogisticRegression       0.997038  0.002136      0.995272  0.004095   

                             Best CV Score            
                                      mean       std  
Test Size Classifier                                  
19-80     LogisticRegression      0.983408  0.004116  
50-50     LogisticRegression      0.993997  0.002736  
80-20     LogisticRegression      0.992890  0.001185  
Confusion Matrix for Test Size 80-20 and Seed 0:
[[118   0]
 [  1 304]]
Confusion Matrix for Test Size 80-20 and Seed 1:
[[101   1]
 [  0 321]]
Confusion Matrix fo

,Test Size,Random Seed,Best Params,Best CV Score,Train Accuracy,Test Accuracy,Classifier,Macro Precision,Macro Recall,Macro F1-Score,Confusion Matrix
0,80-20,0,"{'classifier__C': 1, 'classifier__penalty': 'l...",0.991705,0.994668,0.997636,LogisticRegression,0.995798,0.998361,0.997069,"[[118, 0], [1, 304]]"
1,80-20,1,"{'classifier__C': 10, 'classifier__penalty': '...",0.992889,0.997630,0.997636,LogisticRegression,0.998447,0.995098,0.996759,"[[101, 1], [0, 321]]"
2,80-20,2,"{'classifier__C': 10, 'classifier__penalty': '...",0.994076,0.998815,0.990544,LogisticRegression,0.990636,0.984884,0.987714,"[[108, 3], [1, 311]]"
3,50-50,0,"{'classifier__C': 10, 'classifier__penalty': '...",0.992417,0.997156,0.991477,LogisticRegression,0.989740,0.988661,0.989199,"[[281, 5], [4, 766]]"
4,50-50,1,"{'classifier__C': 1, 'classifier__penalty': 'l...",0.992417,0.996209,0.990530,LogisticRegression,0.989834,0.984848,0.987309,"[[257, 7], [3, 789]]"
5,50-50,2,"{'classifier__C': 10, 'classifier__penalty': '...",0.997156,0.999052,0.992424,LogisticRegression,0.993703,0.987071,0.990324,"[[278, 7], [1, 770]]"
6,19-80,0,"{'classifier__C': 10, 'classifier__penalty': '...",0.985770,1.000000,0.985198,LogisticRegression,0.982283,0.980307,0.981289,"[[446, 14], [11, 1218]]"
7,19-80,1,"{'classifier__C': 1, 'classifier__penalty': 'l...",0.978655,0.988152,0.987567,LogisticRegression,0.987797,0.979697,0.983658,"[[421, 16], [5, 1247]]"
8,19-80,2,"{'classifier__C': 10, 'classifier__penalty': '...",0.985798,1.000000,0.990527,LogisticRegression,0.989251,0.986468,0.987848,"[[440, 10], [6, 1233]]"


## KNN


In [13]:
from sklearn.neighbors import KNeighborsClassifier

In [14]:
knn_param_grid = {
    # K from 1 to 104 with step 4
    'classifier__n_neighbors': np.arange(1, 105, 4),
    # 'uniform' or 'distance' weights
    'classifier__weights': ['uniform', 'distance'],
    'classifier__metric': ['euclidean', 'manhattan'],  # Distance metrics
}

# Assuming 'comprehensive_model_evaluation' is a function for evaluation
knn_results = comprehensive_model_evaluation(
    X, y,
    KNeighborsClassifier(),
    knn_param_grid
)

knn_results

Split Ratios:   0%|          | 0/3 [00:00<?, ?it/s]

Split Ratios: 100%|██████████| 3/3 [00:16<00:00,  5.46s/it]

Evaluation Summary:
                               Train Accuracy      Test Accuracy            \
                                         mean  std          mean       std   
Test Size Classifier                                                         
19-80     KNeighborsClassifier            1.0  0.0      0.911979  0.011113   
50-50     KNeighborsClassifier            1.0  0.0      0.938447  0.003280   
80-20     KNeighborsClassifier            1.0  0.0      0.942474  0.009842   

                               Best CV Score            
                                        mean       std  
Test Size Classifier                                    
19-80     KNeighborsClassifier      0.922633  0.016864  
50-50     KNeighborsClassifier      0.929542  0.007177  
80-20     KNeighborsClassifier      0.940164  0.002136  
Confusion Matrix for Test Size 80-20 and Seed 0:
[[104  14]
 [  7 298]]
Confusion Matrix for Test Size 80-20 and Seed 1:
[[ 83  19]
 [  4 317]]
Confusion Matrix for Test

,Test Size,Random Seed,Best Params,Best CV Score,Train Accuracy,Test Accuracy,Classifier,Macro Precision,Macro Recall,Macro F1-Score,Confusion Matrix
0,80-20,0,"{'classifier__metric': 'manhattan', 'classifie...",0.937800,1.0,0.950355,KNeighborsClassifier,0.946033,0.929203,0.937131,"[[104, 14], [7, 298]]"
1,80-20,1,"{'classifier__metric': 'manhattan', 'classifie...",0.941954,1.0,0.945626,KNeighborsClassifier,0.948738,0.900632,0.921650,"[[83, 19], [4, 317]]"
2,80-20,2,"{'classifier__metric': 'manhattan', 'classifie...",0.940737,1.0,0.931442,KNeighborsClassifier,0.926876,0.892585,0.907859,"[[90, 21], [8, 304]]"
3,50-50,0,"{'classifier__metric': 'manhattan', 'classifie...",0.934597,1.0,0.936553,KNeighborsClassifier,0.931904,0.904845,0.917154,"[[239, 47], [20, 750]]"
4,50-50,1,"{'classifier__metric': 'manhattan', 'classifie...",0.932701,1.0,0.936553,KNeighborsClassifier,0.931604,0.895833,0.911853,"[[215, 49], [18, 774]]"
5,50-50,2,"{'classifier__metric': 'manhattan', 'classifie...",0.921327,1.0,0.942235,KNeighborsClassifier,0.944223,0.907359,0.923650,"[[237, 48], [13, 758]]"
6,19-80,0,"{'classifier__metric': 'manhattan', 'classifie...",0.940924,1.0,0.899941,KNeighborsClassifier,0.897912,0.842829,0.864920,"[[330, 130], [39, 1190]]"
7,19-80,1,"{'classifier__metric': 'manhattan', 'classifie...",0.919272,1.0,0.921847,KNeighborsClassifier,0.922062,0.869825,0.891712,"[[333, 104], [28, 1224]]"
8,19-80,2,"{'classifier__metric': 'manhattan', 'classifie...",0.907703,1.0,0.914150,KNeighborsClassifier,0.907090,0.867899,0.884849,"[[346, 104], [41, 1198]]"


## Decision Tree


In [15]:
from sklearn.tree import DecisionTreeClassifier

In [16]:
dt_param_grid = {
    # Maximum depth of the tree
    'classifier__max_depth': [None, 5, 10, 20, 30],
    # Minimum number of samples required to split a node
    'classifier__min_samples_split': [2, 5, 10],
    # Minimum number of samples required to be at a leaf node
    'classifier__min_samples_leaf': [1, 2, 4],
    # The function to measure the quality of a split
    'classifier__criterion': ['gini', 'entropy'],
    # Strategy used to split at each node
    'classifier__splitter': ['best', 'random'],
    # The number of features to consider for the best split
    'classifier__max_features': [None, 'sqrt', 'log2']
}

# Model evaluation using comprehensive_model_evaluation
dt_results = comprehensive_model_evaluation(
    X, y,
    DecisionTreeClassifier(),
    dt_param_grid
)

dt_results

Split Ratios: 100%|██████████| 3/3 [01:02<00:00, 20.81s/it]

Evaluation Summary:
                                 Train Accuracy          Test Accuracy  \
                                           mean      std          mean   
Test Size Classifier                                                     
19-80     DecisionTreeClassifier       0.992891  0.00627      0.951253   
50-50     DecisionTreeClassifier       1.000000  0.00000      0.976326   
80-20     DecisionTreeClassifier       1.000000  0.00000      0.985028   

                                           Best CV Score            
                                       std          mean       std  
Test Size Classifier                                                
19-80     DecisionTreeClassifier  0.013057      0.959701  0.013226  
50-50     DecisionTreeClassifier  0.004128      0.973776  0.004274  
80-20     DecisionTreeClassifier  0.004921      0.980849  0.001241  
Confusion Matrix for Test Size 80-20 and Seed 0:
[[114   4]
 [  3 302]]
Confusion Matrix for Test Size 80-20 and Seed 1:


,Test Size,Random Seed,Best Params,Best CV Score,Train Accuracy,Test Accuracy,Classifier,Macro Precision,Macro Recall,Macro F1-Score,Confusion Matrix
0,80-20,0,"{'classifier__criterion': 'entropy', 'classifi...",0.982238,1.000000,0.983452,DecisionTreeClassifier,0.980644,0.978133,0.979378,"[[114, 4], [3, 302]]"
1,80-20,1,"{'classifier__criterion': 'entropy', 'classifi...",0.979852,1.000000,0.981087,DecisionTreeClassifier,0.977260,0.970817,0.973987,"[[97, 5], [3, 318]]"
2,80-20,2,"{'classifier__criterion': 'entropy', 'classifi...",0.980456,1.000000,0.990544,DecisionTreeClassifier,0.990636,0.984884,0.987714,"[[108, 3], [1, 311]]"
3,50-50,0,"{'classifier__criterion': 'entropy', 'classifi...",0.973460,1.000000,0.973485,DecisionTreeClassifier,0.976078,0.956543,0.965743,"[[263, 23], [5, 765]]"
4,50-50,1,"{'classifier__criterion': 'gini', 'classifier_...",0.969668,1.000000,0.974432,DecisionTreeClassifier,0.965367,0.966540,0.965952,"[[251, 13], [14, 778]]"
5,50-50,2,"{'classifier__criterion': 'entropy', 'classifi...",0.978199,1.000000,0.981061,DecisionTreeClassifier,0.982457,0.969336,0.975644,"[[269, 16], [4, 767]]"
6,19-80,0,"{'classifier__criterion': 'entropy', 'classifi...",0.973950,0.997630,0.940201,DecisionTreeClassifier,0.927836,0.920143,0.923885,"[[403, 57], [44, 1185]]"
7,19-80,1,"{'classifier__criterion': 'entropy', 'classifi...",0.957339,0.995261,0.965660,DecisionTreeClassifier,0.954631,0.955983,0.955304,"[[409, 28], [30, 1222]]"
8,19-80,2,"{'classifier__criterion': 'gini', 'classifier_...",0.947815,0.985782,0.947898,DecisionTreeClassifier,0.928179,0.941138,0.934362,"[[417, 33], [55, 1184]]"


## Bagging Classifier


In [17]:
from sklearn.ensemble import BaggingClassifier

In [18]:
bagging_param_grid = {
    # Number of base estimators (trees)
    'classifier__n_estimators': [10, 50, 100, 200],
    # Proportion of samples to train each base estimator
    'classifier__max_samples': [0.5, 0.7, 1.0],
    # Proportion of features to train each base estimator
    'classifier__max_features': [0.5, 0.7, 1.0],
    # Whether to use bootstrap sampling
    'classifier__bootstrap': [True, False],
    # Base estimator (Decision Tree)
    'classifier__estimator': [DecisionTreeClassifier(random_state=42)],
    'classifier__random_state': [42]  # For reproducibility
}

# Model evaluation using comprehensive_model_evaluation
bagging_results = comprehensive_model_evaluation(
    X, y,
    BaggingClassifier(),
    bagging_param_grid
)

Split Ratios: 100%|██████████| 3/3 [02:07<00:00, 42.56s/it]

Evaluation Summary:
                            Train Accuracy           Test Accuracy            \
                                      mean       std          mean       std   
Test Size Classifier                                                           
19-80     BaggingClassifier        0.99921  0.001368      0.967634  0.011905   
50-50     BaggingClassifier        1.00000  0.000000      0.980429  0.004270   
80-20     BaggingClassifier        1.00000  0.000000      0.986604  0.003611   

                            Best CV Score            
                                     mean       std  
Test Size Classifier                                 
19-80     BaggingClassifier      0.963679  0.013558  
50-50     BaggingClassifier      0.981359  0.001095  
80-20     BaggingClassifier      0.985782  0.000588  
Confusion Matrix for Test Size 80-20 and Seed 0:
[[114   4]
 [  2 303]]
Confusion Matrix for Test Size 80-20 and Seed 1:
[[ 96   6]
 [  1 320]]
Confusion Matrix for Test Size 

In [19]:
bagging_results

,Test Size,Random Seed,Best Params,Best CV Score,Train Accuracy,Test Accuracy,Classifier,Macro Precision,Macro Recall,Macro F1-Score,Confusion Matrix
0,80-20,0,"{'classifier__bootstrap': True, 'classifier__e...",0.985197,1.00000,0.985816,BaggingClassifier,0.984865,0.979772,0.982278,"[[114, 4], [2, 303]]"
1,80-20,1,"{'classifier__bootstrap': True, 'classifier__e...",0.986373,1.00000,0.983452,BaggingClassifier,0.985643,0.969031,0.977002,"[[96, 6], [1, 320]]"
2,80-20,2,"{'classifier__bootstrap': True, 'classifier__e...",0.985778,1.00000,0.990544,BaggingClassifier,0.990636,0.984884,0.987714,"[[108, 3], [1, 311]]"
3,50-50,0,"{'classifier__bootstrap': False, 'classifier__...",0.981991,1.00000,0.976326,BaggingClassifier,0.975749,0.963986,0.969659,"[[268, 18], [7, 763]]"
4,50-50,1,"{'classifier__bootstrap': True, 'classifier__e...",0.981991,1.00000,0.984848,BaggingClassifier,0.979798,0.979798,0.979798,"[[256, 8], [8, 784]]"
5,50-50,2,"{'classifier__bootstrap': False, 'classifier__...",0.980095,1.00000,0.980114,BaggingClassifier,0.976291,0.973111,0.974685,"[[273, 12], [9, 762]]"
6,19-80,0,"{'classifier__bootstrap': True, 'classifier__e...",0.978768,0.99763,0.954411,BaggingClassifier,0.950431,0.933307,0.941394,"[[408, 52], [25, 1204]]"
7,19-80,1,"{'classifier__bootstrap': True, 'classifier__e...",0.952521,1.00000,0.970989,BaggingClassifier,0.966521,0.957342,0.961811,"[[406, 31], [18, 1234]]"
8,19-80,2,"{'classifier__bootstrap': True, 'classifier__e...",0.959748,1.00000,0.977501,BaggingClassifier,0.974483,0.967684,0.971015,"[[426, 24], [14, 1225]]"


## Random Forest


In [20]:
from sklearn.ensemble import RandomForestClassifier

In [21]:
rf_param_grid = {
    'classifier__n_estimators': [1024],  # Number of trees set to 1024
    # Various options for max_features
    'classifier__max_features': [1, 2, 4, 6, 8, 12, 16, 20],
    'classifier__random_state': [42]  # For reproducibility
}

# Model evaluation using comprehensive_model_evaluation
rf_results = comprehensive_model_evaluation(
    X, y,
    RandomForestClassifier(),
    rf_param_grid
)

rf_results

Split Ratios: 100%|██████████| 3/3 [01:54<00:00, 38.05s/it]

Evaluation Summary:
                                 Train Accuracy      Test Accuracy            \
                                           mean  std          mean       std   
Test Size Classifier                                                           
19-80     RandomForestClassifier            1.0  0.0      0.969410  0.019152   
50-50     RandomForestClassifier            1.0  0.0      0.980114  0.008417   
80-20     RandomForestClassifier            1.0  0.0      0.984240  0.008302   

                                 Best CV Score            
                                          mean       std  
Test Size Classifier                                      
19-80     RandomForestClassifier      0.959748  0.014490  
50-50     RandomForestClassifier      0.978515  0.001973  
80-20     RandomForestClassifier      0.983214  0.000898  
Confusion Matrix for Test Size 80-20 and Seed 0:
[[111   7]
 [  3 302]]
Confusion Matrix for Test Size 80-20 and Seed 1:
[[ 96   6]
 [  1 320]]
C

,Test Size,Random Seed,Best Params,Best CV Score,Train Accuracy,Test Accuracy,Classifier,Macro Precision,Macro Recall,Macro F1-Score,Confusion Matrix
0,80-20,0,"{'classifier__max_features': 16, 'classifier__...",0.982234,1.0,0.976359,RandomForestClassifier,0.975515,0.965421,0.970305,"[[111, 7], [3, 302]]"
1,80-20,1,"{'classifier__max_features': 20, 'classifier__...",0.983407,1.0,0.983452,RandomForestClassifier,0.985643,0.969031,0.977002,"[[96, 6], [1, 320]]"
2,80-20,2,"{'classifier__max_features': 16, 'classifier__...",0.983999,1.0,0.992908,RandomForestClassifier,0.995238,0.986486,0.990758,"[[108, 3], [0, 312]]"
3,50-50,0,"{'classifier__max_features': 16, 'classifier__...",0.980095,1.0,0.970644,RandomForestClassifier,0.970590,0.954595,0.962205,"[[263, 23], [8, 762]]"
4,50-50,1,"{'classifier__max_features': 16, 'classifier__...",0.979147,1.0,0.982955,RandomForestClassifier,0.983398,0.970960,0.976979,"[[250, 14], [4, 788]]"
5,50-50,2,"{'classifier__max_features': 16, 'classifier__...",0.976303,1.0,0.986742,RandomForestClassifier,0.984225,0.982074,0.983142,"[[277, 8], [6, 765]]"
6,19-80,0,"{'classifier__max_features': 16, 'classifier__...",0.976415,1.0,0.947898,RandomForestClassifier,0.942207,0.924751,0.932974,"[[402, 58], [30, 1199]]"
7,19-80,1,"{'classifier__max_features': 20, 'classifier__...",0.950140,1.0,0.984606,RandomForestClassifier,0.982040,0.977700,0.979844,"[[421, 16], [10, 1242]]"
8,19-80,2,"{'classifier__max_features': 20, 'classifier__...",0.952689,1.0,0.975725,RandomForestClassifier,0.973199,0.964350,0.968659,"[[423, 27], [14, 1225]]"


In [ ]:
combined_results = pd.concat([
    lr_results,
    knn_results,
    dt_results,
    bagging_results,
    rf_results
])

# Aggregate and compare key metrics
comparison_summary = combined_results.groupby('Classifier')[
    ['Test Accuracy', 'Macro F1-Score', 'Best CV Score']
].agg(['mean', 'std'])

In [23]:
comparison_summary

Test Accuracy           Macro F1-Score            \
                                mean       std           mean       std   
Classifier                                                                
BaggingClassifier           0.978222  0.010652       0.971706  0.013656   
DecisionTreeClassifier      0.970869  0.016839       0.962441  0.021161   
KNeighborsClassifier        0.930967  0.016236       0.906753  0.022421   
LogisticRegression          0.991504  0.004095       0.989019  0.005255   
RandomForestClassifier      0.977921  0.013061       0.971319  0.016644   

                       Best CV Score            
                                mean       std  
Classifier                                      
BaggingClassifier           0.976940  0.012204  
DecisionTreeClassifier      0.971442  0.011645  
KNeighborsClassifier        0.930779  0.011983  
LogisticRegression          0.990098  0.005645  
RandomForestClassifier      0.973826  0.013011

In [24]:
comparison_summary.to_excel('obesit_comparison_summary.xlsx')

In [26]:
avg_performance = (
    combined_results.groupby(['Test Size', 'Classifier'])[
        ['Best CV Score', 'Train Accuracy', 'Test Accuracy',
            'Macro Precision', 'Macro Recall', 'Macro F1-Score']
    ]
    .mean()
    .reset_index()
)

# Sorting results for better readability
avg_performance = avg_performance.sort_values(
    by=['Classifier']).reset_index(drop=True)

avg_performance

,Test Size,Classifier,Best CV Score,Train Accuracy,Test Accuracy,Macro Precision,Macro Recall,Macro F1-Score
0,19-80,BaggingClassifier,0.963679,0.999210,0.967634,0.963812,0.952778,0.958073
1,50-50,BaggingClassifier,0.981359,1.000000,0.980429,0.977279,0.972298,0.974714
2,80-20,BaggingClassifier,0.985782,1.000000,0.986604,0.987048,0.977896,0.982331
3,19-80,DecisionTreeClassifier,0.959701,0.992891,0.951253,0.936882,0.939088,0.937850
4,50-50,DecisionTreeClassifier,0.973776,1.000000,0.976326,0.974634,0.964140,0.969113
5,80-20,DecisionTreeClassifier,0.980849,1.000000,0.985028,0.982846,0.977945,0.980360
6,19-80,KNeighborsClassifier,0.922633,1.000000,0.911979,0.909021,0.860184,0.880494
7,50-50,KNeighborsClassifier,0.929542,1.000000,0.938447,0.935911,0.902679,0.917552
8,80-20,KNeighborsClassifier,0.940164,1.000000,0.942474,0.940549,0.907473,0.922213
9,19-80,LogisticRegression,0.983408,0.996051,0.987764,0.986444,0.982157,0.984265


In [27]:
avg_performance.to_excel('health_average_performance.xlsx')

,Test Size,Random Seed,Best Params,Best CV Score,Train Accuracy,Test Accuracy,Classifier,Macro Precision,Macro Recall,Macro F1-Score,Confusion Matrix
0,80-20,0,"{'classifier__C': 1, 'classifier__penalty': 'l...",0.991705,0.994668,0.997636,LogisticRegression,0.995798,0.998361,0.997069,"[[118, 0], [1, 304]]"
1,80-20,1,"{'classifier__C': 10, 'classifier__penalty': '...",0.992889,0.997630,0.997636,LogisticRegression,0.998447,0.995098,0.996759,"[[101, 1], [0, 321]]"
2,80-20,2,"{'classifier__C': 10, 'classifier__penalty': '...",0.994076,0.998815,0.990544,LogisticRegression,0.990636,0.984884,0.987714,"[[108, 3], [1, 311]]"
3,50-50,0,"{'classifier__C': 10, 'classifier__penalty': '...",0.992417,0.997156,0.991477,LogisticRegression,0.989740,0.988661,0.989199,"[[281, 5], [4, 766]]"
4,50-50,1,"{'classifier__C': 1, 'classifier__penalty': 'l...",0.992417,0.996209,0.990530,LogisticRegression,0.989834,0.984848,0.987309,"[[257, 7], [3, 789]]"
5,50-50,2,"{'classifier__C': 10, 'classifier__penalty': '...",0.997156,0.999052,0.992424,LogisticRegression,0.993703,0.987071,0.990324,"[[278, 7], [1, 770]]"
6,19-80,0,"{'classifier__C': 10, 'classifier__penalty': '...",0.985770,1.000000,0.985198,LogisticRegression,0.982283,0.980307,0.981289,"[[446, 14], [11, 1218]]"
7,19-80,1,"{'classifier__C': 1, 'classifier__penalty': 'l...",0.978655,0.988152,0.987567,LogisticRegression,0.987797,0.979697,0.983658,"[[421, 16], [5, 1247]]"
8,19-80,2,"{'classifier__C': 10, 'classifier__penalty': '...",0.985798,1.000000,0.990527,LogisticRegression,0.989251,0.986468,0.987848,"[[440, 10], [6, 1233]]"
0,80-20,0,"{'classifier__metric': 'manhattan', 'classifie...",0.937800,1.000000,0.950355,KNeighborsClassifier,0.946033,0.929203,0.937131,"[[104, 14], [7, 298]]"


In [29]:
best_params = combined_results.groupby(
    'Classifier')['Best Params'].apply(list).reset_index()

best_params

,Classifier,Best Params
0,BaggingClassifier,"[{'classifier__bootstrap': True, 'classifier__..."
1,DecisionTreeClassifier,"[{'classifier__criterion': 'entropy', 'classif..."
2,KNeighborsClassifier,"[{'classifier__metric': 'manhattan', 'classifi..."
3,LogisticRegression,"[{'classifier__C': 1, 'classifier__penalty': '..."
4,RandomForestClassifier,"[{'classifier__max_features': 16, 'classifier_..."
